#Question 2
Print the top 5 symbols with quote asset USDT and the highest number of trades over the last 24 hours in descending order.
   
#Question 4
What is the price spread for each of the symbols from Q2?

In [2]:
import requests
import os
from dotenv import load_dotenv
import json
import pandas as pd
import time
import datetime
load_dotenv()

True

In [ ]:
df=pd.read_csv(f"{os.getenv('dataCsvPath')}\{os.getenv('symbolCsvFileName')}.csv", index_col=None)
print(df.loc[df['quoteAsset'] == 'USDT'])

In [14]:
symboldf=pd.read_csv(f"{os.getenv('dataCsvPath')}\{os.getenv('symbolCsvFileName')}.csv", index_col=None)
usdtQuoteAssetdf= symboldf.loc[symboldf['quoteAsset'] == "USDT"]

In [ ]:
# Get 24hr ticker with volume

result=[]

for i, row in  usdtQuoteAssetdf.iterrows():

    tradesUrl=f"{os.getenv('host')}/{os.getenv('tradesPathParam')}?interval=1d&symbol={row['symbol']}"
    print(tradesUrl)
    r = requests.get(tradesUrl)
    df=pd.DataFrame.from_records(r.json(), columns=['OpenTime','Open','High','Low','Close','Volume','Closetime','QuoteAssetVolume','NumberOfTrades','TakerBuyBaseAssetVolume','TakerBuyQuoteAssetVolume', 'ignore'])

    min=float(df.min().Low)
    max=float(df.min().High)

    total_trades = df['NumberOfTrades'].agg('sum')
    print
    d={}
    d['symbol']=row['symbol']
    d['quoteAsset']=row['quoteAsset']
    d["numberOfTrades"]=total_trades
    d['min']=min
    d['max']=max
    d['priceSpread']=max-min
    result.append(d)

df = pd.DataFrame.from_records(result)
df.to_csv(f"{os.getenv('dataCsvPath')}\{os.getenv('allTradewithSymbol')}.csv", index=False)

In [ ]:
df=pd.read_csv(f"{os.getenv('dataCsvPath')}\{os.getenv('allTradewithSymbol')}.csv", index_col=None)
df2=df.sort_values(by=['numberOfTrades'], ascending=False).head(5)
display(df2)
df2.to_csv(f"{os.getenv('dataCsvPath')}\{os.getenv('q2_TradewithSymbol')}.csv", index=False)

In [17]:
def substract(x):
    return x['max'] - x['min']

In [18]:
df3=df2.assign(priceSpread=lambda x: substract(x))
df4=df3[['symbol', 'priceSpread']]
df4.to_csv(f"{os.getenv('dataCsvPath')}\{os.getenv('q4_TradewithSymbolPriceSpread')}.csv", index=False)